<a href="https://colab.research.google.com/github/komh9575/first-repository/blob/main/%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C%20%EC%82%AC%EA%B8%B0%20%EA%B1%B0%EB%9E%98%20%EC%A1%B0%EC%82%AC%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('/content/sample_data/fraud.csv')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 491134 entries, 0 to 491133
Data columns (total 22 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   trans_date_trans_time  491134 non-null  object 
 1   cc_num                 491134 non-null  int64  
 2   merchant               491134 non-null  object 
 3   category               491134 non-null  object 
 4   amt                    491134 non-null  float64
 5   first                  491134 non-null  object 
 6   last                   491134 non-null  object 
 7   gender                 491134 non-null  object 
 8   street                 491134 non-null  object 
 9   city                   491134 non-null  object 
 10  state                  491134 non-null  object 
 11  zip                    491134 non-null  int64  
 12  lat                    491134 non-null  float64
 13  long                   491134 non-null  float64
 14  city_pop               491134 non-nu

In [5]:
#신용카드 번호 제거 (불필요하므로)
df = df.drop(columns=['cc_num'])


In [6]:
# 결제금액 분포가 비대칭적이면 위험 인자
# 로그변환은 큰 값을 적당히 줄이고 작은 값은 거의 그대로 둔다
# 사기 거래는 고액일 때 급증하기 때문에
# 원래 amt, log_amt, is_high_amount 세 가지를 모두 입력하면 \
# 모델이 금액 자체의 크기, 고액 여부의 패턴, 비대칭 문제 해결
# 사기 패턴을 훨씬 더 정교하게 잡아낼 수 있다.

df['log_amt'] = np.log1p(df['amt'])
df['is_high_amount'] = (df['amt'] > 5000).astype(int)


In [7]:
# 고객의 위치와 가맹점 위치가 서로 비슷해야 사기 확률이 적음
# 피타고라스 정리 (거리 공식)

df['distance'] = np.sqrt(
    (df['lat'] - df['merch_lat'])**2 +
    (df['long'] - df['merch_long'])**2
)



In [8]:
#문자인 값 삭제
cols_to_drop = ['dob', 'trans_num']
df = df.drop(columns=[c for c in cols_to_drop if c in df.columns])

In [9]:
object_cols = [
    col for col in df.select_dtypes(include='object').columns
    if col != 'trans_date_trans_time'
]

In [10]:
df = pd.get_dummies(df, columns=object_cols, drop_first=True)


In [11]:
X = df.drop(['is_fraud', 'trans_date_trans_time'], axis=1)
y = df['is_fraud']

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

In [1]:
y_hat = knn.predict(X_test)
print(y_hat[:10])
print(y_test.values[:10])

NameError: name 'knn' is not defined